In [1]:
import torch
import os 
import sys
import umap, numba
import numpy as np
import tqdm
import glob, pandas as pd
import pickle
import fire
import scipy.sparse as sps
from visdom import Visdom
from sklearn.metrics import f1_score
import copy
import torch.nn.functional as F
from torch import nn
from sklearn.preprocessing import LabelEncoder
import torch_cluster
from torch_cluster import knn_graph
from torch_geometric.data import Data 
from torch_geometric.utils import train_test_split_edges
from torch_geometric.utils.convert import to_networkx
from torch_geometric.data import InMemoryDataset,DataLoader
from torch_cluster import radius_graph
from torch_geometric.utils import subgraph

2022-10-20 17:29:09.172882: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
IDs=['109_A1c_ASAP_tumor_map', '10_A1a_ASAP_tumor_map', '10_A1b_ASAP_tumor_map', '10_A2b_ASAP_tumor_map', '110_A2b_ASAP_tumor_map', '112_a_ASAP_tumor_map', '112_b_ASAP_tumor_map', '123_A1a_ASAP_tumor_map', '12_A1c_ASAP_tumor_map', '14_A1b_ASAP_tumor_map', '14_A2b_ASAP_tumor_map', '169_A2b_ASAP_tumor_map', '270_A1b_ASAP_tumor_map', '270_A1d_ASAP_tumor_map', '270_A1e_ASAP_tumor_map', '270_A2b_ASAP_tumor_map', '270_A2f_ASAP_tumor_map', '281_A1d_ASAP_tumor_map', '281_A1f_ASAP_tumor_map', '281_A2eX_ASAP_tumor_map', '311_A2c_ASAP_tumor_map', '327_A1a_ASAP_tumor_map', '327_A1d_ASAP_tumor_map', '327_B1c_ASAP_tumor_map', '341_a_ASAP_tumor_map', '341_b_ASAP_tumor_map', '342_a_ASAP_tumor_map', '342_b_ASAP_tumor_map', '343_a_ASAP_tumor_map', '343_b_ASAP_tumor_map', '343_c_ASAP_tumor_map', '343_d_ASAP_tumor_map', '344_a_ASAP_tumor_map', '344_b_ASAP_tumor_map', '345_a_ASAP_tumor_map', '345_b_ASAP_tumor_map', '346_a_ASAP_tumor_map', '346_b_ASAP_tumor_map', '350_A1a_ASAP_tumor_map', '350_A1b_ASAP_tumor_map', '350_A1c_ASAP_tumor_map', '350_A1d_ASAP_tumor_map', '350_A1e_ASAP_tumor_map', '351_A2b_ASAP_tumor_map', '352_A1d_ASAP_tumor_map', '352_A1e_ASAP_tumor_map', '352_A1g_ASAP_tumor_map', '352_A1h_ASAP_tumor_map', '352_A1i_ASAP_tumor_map', '353_A2b_ASAP_tumor_map', '354_A1b_ASAP_tumor_map', '354_A1c_ASAP_tumor_map', '354_A1d_ASAP_tumor_map', '354_A3a_ASAP_tumor_map', '354_A3b_ASAP_tumor_map', '354_A3c_ASAP_tumor_map', '354_D1b_ASAP_tumor_map', '355_A1d_ASAP_tumor_map', '356_A1b_ASAP_tumor_map', '358_A1a_ASAP_tumor_map', '358_A1b_ASAP_tumor_map', '361_a_ASAP_tumor_map', '361_b_ASAP_tumor_map', '362_A1a_ASAP_tumor_map', '362_A1b_ASAP_tumor_map', '362_A1c_ASAP_tumor_map', '363_A1b_ASAP_tumor_map', '363_A1c_ASAP_tumor_map', '363_A2b_ASAP_tumor_map', '363_A3b_ASAP_tumor_map', '364_A1b_ASAP_tumor_map', '364_A2b_ASAP_tumor_map', '364_A4b_ASAP_tumor_map', '365_A1b_ASAP_tumor_map', '365_A2b_ASAP_tumor_map', '366_A1a_ASAP_tumor_map', '366_A1b_ASAP_tumor_map', '366_A1c_ASAP_tumor_map', '367_A2b_ASAP_tumor_map', '368_A1b_ASAP_tumor_map', '368_A1c_ASAP_tumor_map', '368_A1d_ASAP_tumor_map', '369_A1b_ASAP_tumor_map', '369_A1c_ASAP_tumor_map', '369_A2b_ASAP_tumor_map', '370_A1b_ASAP_tumor_map', '370_A2a_ASAP_tumor_map', '370_A2b_ASAP_tumor_map', '37_A2d_ASAP_tumor_map', '61_A1a_ASAP_tumor_map', '61_B1a_ASAP_tumor_map', '70_A2b_ASAP_tumor_map', '7_A1c_ASAP_tumor_map', '7_A1d_ASAP_tumor_map', '7_A1e_ASAP_tumor_map']

In [3]:

#TUMOR_PERCENT=0.2

def get_graph_dataset(embedding_dir='/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/cnn_embeddings/', 
                      basename='', 
                      target_col='scc',  
                      k=8,
                      radius=0):
    with open(os.path.join(embedding_dir,f"{basename}.pkl"),"rb") as file:
        embeddings=pickle.load(file) 
    df=embeddings['patch_info']
    X=pd.DataFrame(embeddings['embeddings'])
    df['y_true']=df[target_col]
    if len(target_mapping): df['y_true']=df['y_true'].map(target_mapping)
    xy=torch.tensor(embeddings['patch_info'][['x','y']].values).float().cuda()
    X=torch.tensor(X.values)
    y=torch.tensor(df.loc[:,'y_true'].values)
    
    if not radius:
        G=knn_graph(xy,k=k)
    else:
        G=radius_graph(xy, r=radius*np.sqrt(2), batch=None, loop=True)
    G=G.detach().cpu()
    G=torch_geometric.utils.add_remaining_self_loops(G)[0]
    xy=xy.detach().cpu()
    datasets=[]
    edges=G.detach().cpu().numpy().astype(int)
    n_components,components=list(sps.csgraph.connected_components(sps.coo_matrix((np.ones_like(edges[0]),(edges[0],edges[1])))))
    components=torch.LongTensor(components)
    for i in range(n_components):
        G_new=subgraph(components==i,G,relabel_nodes=True)[0]
        xy_new=xy[components==i]
        X_new=X[components==i]
        y_new=y[components==i]
        #y_whole_slide_image_label=y_new.float().mean()>=TUMOR_PERCENT
        np.random.seed(42)
        idx=np.arange(X_new.shape[0])
        idx2=np.arange(X_new.shape[0])
        np.random.shuffle(idx)
        train_idx,val_idx,test_idx=torch.tensor(np.isin(idx2,idx[:int(0.8*len(idx))])),torch.tensor(np.isin(idx2,idx[int(0.8*len(idx)):int(0.9*len(idx))])),torch.tensor(np.isin(idx2,idx[int(0.9*len(idx)):]))
        dataset=Data(x=X_new, edge_index=G_new, edge_attr=None, y=y_new, pos=xy_new, y_wsi=y_whole_slide_image_label)
        dataset.train_mask=train_idx
        dataset.val_mask=val_idx
        dataset.test_mask=test_idx
        datasets.append(dataset)
    components=components.numpy()
    

    return dataset


In [11]:
for ID in IDs:
    get_graph_dataset(embedding_dir='/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/cnn_embeddings/',
                      basename=ID,
                      target_col='scc',
                      k=8,
                      radius=256)

UnpicklingError: A load persistent id instruction was encountered,
but no persistent_load function was specified.

In [1]:
agraergaergaerg=pickle.load('/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/cnn_embeddings/109_A1c_ASAP_tumor_map.pkl')  

NameError: name 'pickle' is not defined

In [6]:
get_graph_dataset(basename='109_A1c_ASAP_tumor_map')

UnpicklingError: A load persistent id instruction was encountered,
but no persistent_load function was specified.

In [14]:
embeddings=pickle.load("/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/cnn_embeddings/109_A1c_ASAP_tumor_map.pkl")

TypeError: 'str' object is not callable